# Homework 3

## FINM 37400 - 2025

### UChicago Financial Mathematics

* Mark Hendricks
* hendricks@uchicago.edu

***

# 1 HBS Case: Fixed-Income Arbitrage in a Financial Crisis (C): Spread and Swap Spread in November 2008

## Simplification of the setup

The date is Nov 4, 2008.

**Treasury bond**
* Suppose the Treasury bond matures exactly 30 years later, on Nov 4, 2038 rather than May 15, 2008. 
* The YTM of this freshly issued treasury is 4.193\% with a semiannual coupon of 4.50\%, same as is given in the case. (So we're just changing the maturity date to simplify things, but keeping the market data.)

**Swap**
* The fixed leg of the swap pays semiannually, with swap rate of 4.2560\%, as given in the case.
* The floating leg of the swap also pays semiannually--not quarterly--such that the payment dates are identical on both legs. Thus, it also resets the floating rate semiannually, not quarterly.
* The floating rate of the swap equals the repo rate used in the trade. Thus, these two rates cancel in the financing of the trade. (No need to consider the TED spread.) 

## Case Clarifications


### Duration Quotes
Bond
* Quote: Val01 of bond is .1746 per bp per $1 face value
* Class terminology: Modified dollar duration is .1746 per $100 face value

Swap
* Quote: DV01 of swap is 1.7mm per 1 billion notional.
* Class terminology: Modified dollar duration is 100(1.7/1000) per $100 face value.

Thus, modified dollar duration for each per 100 face is
* Bond = .1746
* Swap = .1700

### Hedge Ratio

In figuring out the hedge ratio, they set up the hedge per dollar of face value. 

    *so Mills would need to buy face amount $0.97 billion*
    
No, this hedge should be for market value, not face amount given that the case is already using **modified** duration which includes the dirty price.
    

### Maturity Mismatch

The maturity of the bond is August 2038, whereas the date is Nov 2008. Thus, the bond has less than 30 years to maturity, yet he is entering a 30-year swap. 

For simplicity, we imagine the bond is issued in Nov 2008 and thus has maturity of 30 years at the time of the case.

However, then the case quotes for the Nov price and YTM of the bond no longer are accurate. Use one and adjust the other. Namely, we could...
    * use the Nov 4 **YTM** quoted in the case, and re-adjust the the bond.
    * use the Nov 4 **price** quoted in the case, and re-adjust the YTM.
    
We do the former, keep the quoted YTM, assume time-to-maturity of `30`, and recalculate the Nov 2008 price. (It is close to the quoted price in the case.)

***

## 1.0.

Report the price of the 30-year T-bond in Nov 2008. Given the discussion about `Maturity Mismatch`, we are repricing the bond, so it will not exactly equal `105` as reported in the case.

In [66]:
import numpy as np
from scipy.optimize import fsolve

ytm = 4.193 / 100  
coupon_rate = 4.50 / 100
face_value = 100
years_to_maturity = 30
coupons_per_year = 2

def bond_price(estimate):
    periods = years_to_maturity * coupons_per_year
    coupon_payment = (coupon_rate / coupons_per_year) * face_value
    discount_factors = [(1 + ytm / coupons_per_year) ** (-t) for t in range(1, periods + 1)]
    present_value_coupons = sum([coupon_payment * df for df in discount_factors])
    present_value_face = face_value * discount_factors[-1]

    return present_value_coupons + present_value_face - estimate

int_estimate = 105  
bond_price_nov_2008 = fsolve(bond_price, int_estimate)[0]
print("Price ${:,.2f}".format(bond_price_nov_2008))

Price $105.21


## 1.1

List the projected cashflows on May 4, 2009, exactly six months into the trade, on the first coupon and swap date.

#### Clarification
List these cashflows for face value of $1B, not the $0.97B noted in the case. As mentioned in "Case Clarifications", we will not use this number. Rather, we calculate our own hedge ratio in a problem below.

# There are two cash flows:
# 1) Treasury Bond Coupon Payment: $22,500,000 (this is from the 4.50% coupon rate)
# 2) Fixed Swap Payment $21,280,000 (this is from the 4.256% swap rate)

## 1.2

What is the duration of...
* the T-bond
* the swap

Remember that...
* the swap can be decomposed into a fixed-rate bond and a floating-rate note
* a floating-rate note has duration equal to the time until the next reset. Thus, at initialization, it has duration equal to 0.5 years.

Is the duration for the "paying-fixed" swap positive or negative? Is it bigger or smaller in magnitude than the T-bond?

For this problem, calculate the Macauley duration and the dollar (Macauley) duration.

In [67]:

years_to_maturity = 30
coupons_per_year = 2
ytm = 4.193 / 100
swap_rate = 4.256 / 100
notional = 1_000_000_000
coupon_payment = (coupon_rate / coupons_per_year) * face_value

def macauley_duration(ytm, coupon_rate, face_value, years_to_maturity, coupons_per_year):
    periods = years_to_maturity * coupons_per_year
    discount_factors = [(1 + ytm / coupons_per_year) ** (-t) for t in range(1, periods + 1)]
    time_factors = [(t / coupons_per_year) * df for t, df in zip(range(1, periods + 1), discount_factors)]
    
    pv_coupons = sum([coupon_payment * t for coupon_payment, t in zip([coupon_rate / coupons_per_year * face_value] * periods, time_factors)])
    pv_face = face_value * discount_factors[-1] * years_to_maturity
    price = sum([coupon_payment * df for df in discount_factors]) + face_value * discount_factors[-1]

    return (pv_coupons + pv_face) / price

macauley_duration_bond = macauley_duration(ytm, coupon_rate, face_value, years_to_maturity, coupons_per_year)
macauley_duration_fixed_leg = macauley_duration(ytm, swap_rate, face_value, years_to_maturity, coupons_per_year)
macauley_duration_floating_leg = 0.5
macauley_duration_swap = macauley_duration_fixed_leg - macauley_duration_floating_leg

dollar_duration_bond = macauley_duration_bond * (notional / 100)
dollar_duration_swap = macauley_duration_swap * (notional / 100)

In [68]:
print("Durations: \nBond {:.2f} \nSwap {:.2f}".format(macauley_duration_bond, macauley_duration_swap))

Durations: 
Bond 17.08 
Swap 16.10


In [69]:
print("Dollar Durations: \nBond ${:,.0f} \nSwap ${:,.0f}".format(dollar_duration_bond, dollar_duration_swap))


Dollar Durations: 
Bond $170,836,331 
Swap $161,025,427


# The duration of the paying-fixed swap is positive, meaning that its value decreases when interest rates rise.

# The magnitude of the swap's duration is smaller than the T-bond's duration, as the floating leg reduces overall duration.

## 1.3

What hedge ratio should be used to balance the notional size of the Treasury bond with the notional size of the swap, such that it is a duration-neutral position?

Specifically, if the trader enters the swap paying fixed on \$500 million notional, how large of a position should they take in the Treasury bond?

In [70]:
swap_notional = 500_000_000
bond_notional = (swap_notional * macauley_duration_swap) / macauley_duration_bond
bond_notional
print("Position in Treasury bond: ${:,.0f}".format(bond_notional))


Position in Treasury bond: $471,285,663


## 1.4

Suppose it is May 4, 2009, exactly six months after putting the trade on.

The spread is at -28 bps due to...
* The YTM on a new 30-year bond has risen to 4.36\%
* The swap rate on a new 30-year swap has dropped to 4.08\%

Explain conceptually how this movement impacts the components of the trade.

# The trade loses value because both components move against it. The Treasury bond price drops as yields rise, causing a loss on the long bond positio. Meanwhile, the swap loses value as the fixed rate declines, making the pay-fixed swap less favorable. Since the strategy relied on a widening spread, the further compression to -28 bps results in a net loss.

## 1.5

Calculate the value of the position on May 4, 2009, immediately after the first coupon and swap payments and swap reset. 

* Calculate the revised price of the Treasury bond by assuming you can apply the (May 4) 30-year YTM as a discount rate to the 29.5 year bond. (We are just using this for a rough approximation. You know that good pricing would require a discount curve, but let's not get bogged down with that here.)


* Calculate the value of the swap by decomposing it into a fixed-rate bond and a floating-rate bond.
    * The 29.5 year fixed-rate leg is priced using the (May 4) 30-year swap rate as a discount rate.
    * The floating-rate leg is priced at par given that floating-rate notes are par immediately after resets.
    
**Note**

You are being asked to calculate these valuations using the exact formula between price, cashflows, and YTM discount rate. We are not simply approximating with duration, as we already know the position was set up with zero dollar duration.

From the Discussion notebook, we have this formula expressing a bond's price as a function of the coupon, $c$, and the YTM, $y_j$.

$\begin{align*}
P_j(t,T,c) = \sum_{i=1}^{n-1}\frac{100\left(\frac{c}{2}\right)}{\left(1+\frac{y_j}{2}\right)^{2(T_i-t)}} + \frac{100\left(1+\frac{c}{2}\right)}{\left(1+\frac{y_j}{2}\right)^{2(T-t)}}
\end{align*}
$

In [71]:
ytm_may_2009 = 4.36 / 100
swap_rate_may_2009 = 4.08 / 100
years_to_maturity_may_2009 = 29.5

def bond_price_exact(yield_rate, coupon_rate, years_to_maturity, face_value=100, coupons_per_year=2):
    periods = int(years_to_maturity * coupons_per_year)
    coupon_payment = (coupon_rate / coupons_per_year) * face_value
    discount_factors = [(1 + yield_rate / coupons_per_year) ** (-2 * (t / coupons_per_year)) for t in range(1, periods + 1)]
    price = sum([coupon_payment * df for df in discount_factors]) + face_value * discount_factors[-1]
    
    return price

bond_price_may_2009 = bond_price_exact(ytm_may_2009, coupon_rate, years_to_maturity_may_2009)
swap_fixed_leg_price = bond_price_exact(swap_rate_may_2009, swap_rate, years_to_maturity_may_2009)

swap_floating_leg_price = 100
swap_value = swap_fixed_leg_price - swap_floating_leg_price

notional = 500_000_000
bond_position_value = (bond_price_may_2009 / 100) * bond_notional
swap_position_value = (swap_value / 100) * notional


total_position_value = bond_position_value + swap_position_value

print(f"Bond Price (May 2009): ${bond_price_may_2009:,.2f}")
print(f"Swap Fixed Leg Price: ${swap_fixed_leg_price:,.2f}")
print(f"Swap Value: ${swap_value:,.2f}")
print(f"Bond Position Value: ${bond_position_value:,.0f}")
print(f"Swap Position Value: ${swap_position_value:,.0f}")
print(f"Total Position Value: ${total_position_value:,.0f}")


Bond Price (May 2009): $102.31
Swap Fixed Leg Price: $103.00
Swap Value: $3.00
Bond Position Value: $482,178,963
Swap Position Value: $15,016,747
Total Position Value: $497,195,710


## 1.6

Accounting for the change in value of the positions, as well as the 6-month cashflows paid on May 4, 
* what is the net profit and loss (pnl) of the position?
* what is the return on the equity capital, considering that there was a 2\% haircut (equity contribution) on the size of the initial treasury bond position.

In [72]:

initial_bond_position_value = (bond_price_nov_2008 / 100) * bond_notional
initial_swap_position_value = 0
initial_total_position_value = initial_bond_position_value + initial_swap_position_value

total_cashflows_paid = 22_500_000 + 21_280_000 #from previous question

net_pnl = (total_position_value - initial_total_position_value) - total_cashflows_paid

equity_capital = 0.02 * initial_bond_position_value

return_on_equity = net_pnl / equity_capital

print(f"Net PnL: ${net_pnl:,.0f}")
print(f"Return on Equity: {return_on_equity:.2%}")


Net PnL: $-42,439,402
Return on Equity: -427.94%


***

# 2. Factor Duration

### Data

This problem uses data from,
* `/data/yields.xlsx`
* `/data/treasury_ts_duration_2024-10-31.xlsx`

#### Load Yields

In [73]:
import pandas as pd

filepath = '../data/yields.xlsx'
yields = pd.read_excel(filepath, sheet_name='yields')
yields.set_index('caldt',inplace=True)

#### Load Prices and Durations of Two Treasuries

In [74]:
QUOTE_DATE = '2024-10-31'
filepath = f'../data/treasury_ts_duration_{QUOTE_DATE}.xlsx'

data = pd.read_excel(filepath,sheet_name='database')
data_info =  data.drop_duplicates(subset='KYTREASNO', keep='first').set_index('KYTREASNO')
data_info[['type','issue date','maturity date','cpn rate']]

,type,issue date,maturity date,cpn rate
KYTREASNO,,,,
207391,note,2019-08-15,2029-08-15,1.625
207392,bond,2019-08-15,2049-08-15,2.250


You will largely focus on the sheets which give the timeseries of prices and durations for each of the two securities, as shown in the following code.

In [75]:
SHEET_PRICE = 'price'
SHEET_DURATION = 'duration'
INDEX_NAME = 'quote date'

price = pd.read_excel(filepath,sheet_name=SHEET_PRICE).set_index(INDEX_NAME)
duration = pd.read_excel(filepath,sheet_name=SHEET_DURATION).set_index(INDEX_NAME)

### 2.1.

Construct the following yield-curve factors from the `yields` data set:

$\begin{align}
x^{\text{level}}_t =& \frac{1}{N_{\text{yields}}}\sum_{i=1}^{N_{\text{yields}}} y^{(i)}_t\\
x^{\text{slope}}_t =& y^{(30)}_t - y^{(1)}_t\\
x^{\text{curvature}}_t =& -y^{(1)}_t + 2 y^{(10)}_t - y^{(30)}_t
\end{align}$

In [79]:
x_level = yields.mean(axis=1)
x_slope = yields[30] - yields[1]
x_curvature = -yields[1] + 2 * yields[10] - yields[30]

yield_factors = pd.DataFrame({
    'Level': x_level,
    'Slope': x_slope,
    'Curvature': x_curvature
})

yield_factors

,Level,Slope,Curvature
caldt,,,
1961-06-14,3.591027,0.879264,0.886559
1961-06-15,3.614691,0.893380,0.966721
1961-06-16,3.617360,0.900101,0.966566
1961-06-19,3.630646,0.916659,1.013983
1961-06-20,3.627267,0.892599,0.975574
...,...,...,...
2024-05-24,4.642343,-0.637884,-0.838863
2024-05-28,4.711880,-0.536670,-0.764817
2024-05-29,4.768430,-0.551893,-0.787602


### 2.2

Get the bond prices and durations for the two bonds in the data set referenced above.

#### Align the data

Align the bond pricing data with the yield factor data, so that you have data for both in the intersection of their dates.


#### Estimate the regression

Estimate the regression in the form of day-over-day differences for both bond prices and factors. That is, we are using regression to approximate the factor duration equation,

$\begin{align}
\frac{dP}{P} = \alpha + \beta_L dx_{\text{level}} + \beta_S dx_{\text{slope}} + \beta_C dx_{\text{curvature}} + \epsilon
\end{align}$

Report the betas for each of these factors, for each of the bond prices.

In [83]:
import statsmodels.api as sm

aligned_data = price.merge(yield_factors, left_index=True, right_index=True, how='inner')
aligned_data_pct_change = aligned_data.pct_change().dropna()

X = aligned_data_pct_change[['Level', 'Slope', 'Curvature']]
X = sm.add_constant(X)
betas = {}

for bond in price.columns:
    y = aligned_data_pct_change[bond]
    model = sm.OLS(y, X).fit()
    betas[bond] = model.params

betas_df = pd.DataFrame(betas).T

betas_df


,const,Level,Slope,Curvature
207391,0.000062,-0.116812,-0.000012,-0.000023
207392,0.000112,-0.301905,-0.000089,-0.000048


### 2.3.

Compare the "level" factor beta for each of the two treasuries with the average  duration for each bond as reported in the data set.

* How closely does the average duration for a bond compare to its "level" beta?
* What do you conclude about the usefulness of mathematical duration vs regression sensitivities?

In [84]:
avg_duration = duration.mean()

comparison_df = pd.DataFrame({
    'Level Beta': betas_df['Level'],
    'Avg Duration': avg_duration
})

comparison_df

,Level Beta,Avg Duration
207391,-0.116812,6.923664
207392,-0.301905,19.903216


# Mathematical duration is a theoretical measure of bond price sensitivity to interest rates, but the regression betas (level beta) are much smaller, showing that real-world price movements are less sensitive than duration predicts. This difference suggests that factors like convexity, market liquidity, and hedging impact actual bond price behavior. While duration is useful for quick estimates, regression betas provide a more accurate realistic view of interest rate risk.

### 2.4.

In the duration-hedged trade of `Homework 2, Section 2`, was the that trade was long or short this slope factor? 

Do you think the slope factor exposure had a large impact on the trade?

No new analysis needed, just draw a conclusion from the estimates above along with the trade construction in `HW 2, Sec 2`.

# The trade was effectively short the slope factor, meaning it would benefit if the yield curve flattened and lose if it steepened. However, the small magnitude of the slope betas suggests that slope exposure had minimal impact on the trade's performance. Instead, the trade was primarily driven by changes in the level factor.

***

# 3 Calculating Duration Via Individual Cashflows

## *Optional, not submitted*

Use the data file `../data/treasury_quotes_2024-10-31.xlsx`.

### 3.1 
Set up the cashflow matrix. 

### 3.2
Extract the Nelson-Siegel spot discount curve, as you did in `Homework 1`.

### 3.3
For each treasury issue, calculate the duration as the weighted average of the (discounted!) cashflow maturity.

Report the summary statistics of the durations. (Use `.describe()` from pandas.)

### 3.4
How close are your duration estimates to the imputed durations given in the data source?

Report the summary statistics of the imputed durations minus your calculated durations from above.

### 3.5
Continue using your assumed discount rates of `4.5`\% to calculate the convexity of each issue.

Report the summary statistics of these convexity calculations.

***